In [17]:
# WANT: Proj SOG

# Model:

# - SOG/GP TY
# - SOG/GP L3-L5
# - Opp Team Allowed SOG/GP TY
# - Opp Team Allowed SOG/GP L3-L5

# Additional:

# - Any Starting Opp D out
# - Starting Opp G out
# - Back to back games
# - Home vs Away
# - Penalties
# - 


import pandas as pd
fullPageData = pd.read_html('https://www.hockey-reference.com/teams/CGY/')
teamData = fullPageData[1]
rosterData = fullPageData[4]
rosterData.columns = rosterData.columns.droplevel()
print(rosterData.loc[:,["Player", "GP","S"]])

# Average shots per game
avgS = teamData['S'].iloc[0] / teamData['GP'].iloc[0]


print(avgS)
teamData

                Player  GP     S
0        Tyler Toffoli  58   181
1       Elias Lindholm  56   121
2          Nazem Kadri  58   195
3      Mikael Backlund  58   185
4   Jonathan Huberdeau  55    89
5          Dillon Dube  58   105
6     Rasmus Andersson  55   122
7    Andrew Mangiapane  58   132
8        Blake Coleman  58   134
9         Noah Hanifin  57   127
10        Adam Ruzicka  41    59
11    MacKenzie Weegar  57   103
12        Trevor Lewis  58   116
13         Milan Lucic  55    55
14      Nikita Zadorov  58    89
15         Chris Tanev  48    47
16       Michael Stone  46    86
17       Brett Ritchie  34    46
18        Walker Duehr   9    16
19     Jakob Pelletier  12    22
20      Dennis Gilbert  14     9
21       Connor Mackey  10     8
22     Jacob Markstrom  38     0
23        Kevin Rooney  17    14
24       Nick DeSimone   4     4
25    Matthew Phillips   2     1
26       Daniel Vladar  22     0
27       Radim Zohorna   8     3
28          Team Total  58  2069
35.6724137

,Team,AvAge,GP,W,L,OL,PTS,PTS%,GF,GA,...,PPOA,PK%,SH,SHA,S,S%,SA,SV%,PDO,SO
0,Calgary Flames,29.6,58,27,20,11,65,0.560,185,176,...,203,81.28,6,3,2069,8.9,1569,0.888,98.5,0
1,League Average,28.4,57,29,22,6,64,0.556,179,179,...,179,78.37,5,5,1799,9.9,1799,0.901,NaN,2


In [3]:
SGPData = rosterData.loc[:,["Player", "GP","S"]]
SGPData['S/GP'] = SGPData['S']/SGPData['GP']
SGPData.drop(SGPData.tail(1).index,inplace=True)
SGPData['S/GP'] = pd.to_numeric(SGPData['S/GP'])
SGPData = SGPData.loc[SGPData['S/GP'] >= 1.5]
SGPData

,Player,GP,S,S/GP
0,Tyler Toffoli,58,181,3.120690
1,Elias Lindholm,56,121,2.160714
2,Nazem Kadri,58,195,3.362069
3,Mikael Backlund,58,185,3.189655
4,Jonathan Huberdeau,55,89,1.618182
5,Dillon Dube,58,105,1.810345
6,Rasmus Andersson,55,122,2.218182
7,Andrew Mangiapane,58,132,2.275862
8,Blake Coleman,58,134,2.310345
9,Noah Hanifin,57,127,2.228070


In [19]:
import numpy as np

playerL5AVG = []
playerTeamSPerc = []
playerTeamSPercL5 = []
for index, row in SGPData.iterrows():
    player = row['Player']
    sGP = row['S/GP']
    playerTeamSPerc.append((sGP/avgS)*100)
    url = 'https://www.hockey-reference.com/players/'
    nameSplit = player.split()
    url += nameSplit[1][0]
    url += '/'
    url += nameSplit[1][:5]
    url += nameSplit[0][:2]
    url += '01.html'
    url = url.lower()
    playerL5 = pd.read_html(url)
    playerL5 = playerL5[0]
    playerL5.columns = playerL5.columns.droplevel()
    playerL5.drop(playerL5.tail(1).index,inplace=True)
    playerL5 = pd.to_numeric(playerL5.loc[:,"S"])
    if (playerL5.mean() > 6):
        playerL5AVG.append(np.nan)
        playerTeamSPercL5.append(np.nan)
    else:
        playerL5AVG.append(playerL5.mean())
        playerTeamSPercL5.append((playerL5.mean()/avgS)*100)



SGPData["S/GP L5"] = playerL5AVG
SGPData["S/GP Team %"] = playerTeamSPerc
SGPData["S/GP L5 Team %"] = playerTeamSPercL5
SGPData

,Player,GP,S,S/GP,S/GP L5,S/GP Team %,S/GP L5 Team %
0,Tyler Toffoli,58,181,3.120690,3.6,8.748188,10.091832
1,Elias Lindholm,56,121,2.160714,2.0,6.057101,5.606573
2,Nazem Kadri,58,195,3.362069,4.4,9.424843,12.334461
3,Mikael Backlund,58,185,3.189655,4.4,8.941518,12.334461
4,Jonathan Huberdeau,55,89,1.618182,2.0,4.536227,5.606573
5,Dillon Dube,58,105,1.810345,2.2,5.074915,6.167231
6,Rasmus Andersson,55,122,2.218182,2.6,6.218199,7.288545
7,Andrew Mangiapane,58,132,2.275862,3.4,6.379894,9.531174
8,Blake Coleman,58,134,2.310345,1.2,6.476559,3.363944
9,Noah Hanifin,57,127,2.228070,1.0,6.245919,2.803287


TEMPLATE MATH:

Opponents Allowed SOG Avg

Example : 35.23

Avg SOG

Example: 35.4

Toffoli Avg:

~ 8.7% of the SOG for team

8.7% of 35.23 = Proj 3.065 SOG

Toffoli AVG L5:

~ 9.6%

9.6% of 35.23 = 3.382 SOG

AVG OF ALL SOG AVGS:

3.382 + 3.065 + 3.070 + 3.4 = Aggregate project SOG 3.229

In [20]:
oppTeam = "VEG"
url = "https://www.hockey-reference.com/teams/" + oppTeam + "/"
oppFullPageData = pd.read_html(url)
oppTeamData = oppFullPageData[1]
oppRosterData = oppFullPageData[4]
oppRosterData.columns = oppRosterData.columns.droplevel()
oppSA = oppTeamData['SA'].iloc[0] / oppTeamData['GP'].iloc[0]
leagueAvgSA = oppTeamData['SA'].iloc[1] / oppTeamData['GP'].iloc[1]
print(oppSA)
leagueAvgSA

30.157894736842106


31.56140350877193

In [21]:
projSOG = []
projSOGL5 = []
for index, row in SGPData.iterrows():
    sPerc = row['S/GP Team %']/100
    sPercL5 = row['S/GP L5 Team %']/100
    projSOG.append(sPerc*oppSA)
    projSOGL5.append(sPercL5*oppSA)
    
SGPData["Proj S"] = projSOG
SGPData["Proj S L5"] = projSOGL5
SGPData

,Player,GP,S,S/GP,S/GP L5,S/GP Team %,S/GP L5 Team %,Proj S,Proj S L5
0,Tyler Toffoli,58,181,3.120690,3.6,8.748188,10.091832,2.638269,3.043484
1,Elias Lindholm,56,121,2.160714,2.0,6.057101,5.606573,1.826694,1.690824
2,Nazem Kadri,58,195,3.362069,4.4,9.424843,12.334461,2.842334,3.719814
3,Mikael Backlund,58,185,3.189655,4.4,8.941518,12.334461,2.696573,3.719814
4,Jonathan Huberdeau,55,89,1.618182,2.0,4.536227,5.606573,1.368031,1.690824
5,Dillon Dube,58,105,1.810345,2.2,5.074915,6.167231,1.530488,1.859907
6,Rasmus Andersson,55,122,2.218182,2.6,6.218199,7.288545,1.875278,2.198072
7,Andrew Mangiapane,58,132,2.275862,3.4,6.379894,9.531174,1.924042,2.874402
8,Blake Coleman,58,134,2.310345,1.2,6.476559,3.363944,1.953194,1.014495
9,Noah Hanifin,57,127,2.228070,1.0,6.245919,2.803287,1.883638,0.845412


In [22]:
aggProjSOG = []
for index, row in SGPData.iterrows():
    aggProj = row['S/GP'] + row['S/GP L5'] + row['Proj S'] + row['Proj S L5']
    aggProjSOG.append(aggProj/4)
    
SGPData["Agg Proj S"] = aggProjSOG
SGPData

,Player,GP,S,S/GP,S/GP L5,S/GP Team %,S/GP L5 Team %,Proj S,Proj S L5,Agg Proj S
0,Tyler Toffoli,58,181,3.120690,3.6,8.748188,10.091832,2.638269,3.043484,3.100611
1,Elias Lindholm,56,121,2.160714,2.0,6.057101,5.606573,1.826694,1.690824,1.919558
2,Nazem Kadri,58,195,3.362069,4.4,9.424843,12.334461,2.842334,3.719814,3.581054
3,Mikael Backlund,58,185,3.189655,4.4,8.941518,12.334461,2.696573,3.719814,3.501511
4,Jonathan Huberdeau,55,89,1.618182,2.0,4.536227,5.606573,1.368031,1.690824,1.669259
5,Dillon Dube,58,105,1.810345,2.2,5.074915,6.167231,1.530488,1.859907,1.850185
6,Rasmus Andersson,55,122,2.218182,2.6,6.218199,7.288545,1.875278,2.198072,2.222883
7,Andrew Mangiapane,58,132,2.275862,3.4,6.379894,9.531174,1.924042,2.874402,2.618576
8,Blake Coleman,58,134,2.310345,1.2,6.476559,3.363944,1.953194,1.014495,1.619508
9,Noah Hanifin,57,127,2.228070,1.0,6.245919,2.803287,1.883638,0.845412,1.489280
